In [1]:
import torch
import random
import numpy as np
from sklearn.utils import check_random_state
import pandas as pd


random.seed(42)

# Set the random seed for numpy
np.random.seed(42)

# Set the random seed for pandas
# pd.np.random.seed(42)

# Set the random seed for scikit-learn
check_random_state(42)

# Set the random seed for pytorch
torch.manual_seed(42)

In [2]:
from src.load_data import get_data
from src.EchoGAE import EchoGAE_algorithm
from src.echo_chamber_measure import EchoChamberMeasure


# TODO: Work on baselines
from networkx.algorithms.community import asyn_fluidc
from src.baselines.RWC_jit import RWC
# from src.baselines.RWC import RWC # If you don't have numba installed use this line instead but it will be slower

from src.baselines.polarization_index import add_ideology_to_graph, opinion_model, get_polarization_index

In [3]:
datasets = ["gun", "abortion", "super_bowl", "sxsw"]

In [4]:
def add_ideology_to_graph_22(G, ideologies_dict):
    G = G.copy()
    core_nodes = []
    for node in G.nodes():
        if node in ideologies_dict:
            G.nodes[node]["ideology"] = ideologies_dict[node]
            core_nodes.append(node)
        else:
            G.nodes[node]["ideology"] = 0
    return G, core_nodes

In [5]:
experiments = []

use_baseline = True

for ds in datasets:
    print(f"Dataset ({ds}): ", end="")
    
    ds_dict = {}
    ds_dict["dataset"] = ds

    # Get the data
    G, users_embeddings, labels, allsides_scores, node_id_map = get_data(f"data/{ds}/")

    # Graph information
    ds_dict["number_of_nodes"] = G.number_of_nodes()
    ds_dict["number_of_edges"] = G.number_of_edges()
    ds_dict["number_of_communities"] = len(np.unique(labels))


    # My metric
    user_emb = EchoGAE_algorithm(G, user_embeddings= users_embeddings, show_progress=False, hidden_channels = 20, out_channels=10, epochs=300)
    ecm = EchoChamberMeasure(user_emb, labels)
    eci = ecm.echo_chamber_index()
    ds_dict["echo_chamber_score"] = eci

    print(f"ECS = {eci:.3f} -- ", end=" ")

    # For communities ECIs and Sizes
    sizes = []
    ECSs = []

    for i in np.unique(labels):
        sizes.append(np.sum(labels == i))
        ECSs.append(ecm.community_echo_chamber_index(i))

    ds_dict["community_sizes"] = sizes
    ds_dict["community_ECIs"] = ECSs


    if use_baseline:
        # Baseline
        mem = asyn_fluidc(G, k=2, seed=42, max_iter=1000)
        mem = list(mem)

        nodes_0 = np.array(list(mem[0]))
        nodes_1 = np.array(list(mem[1]))


        rwc = RWC(G, nodes_0, nodes_1)
        ds_dict["RWC"] = rwc
        print(f"RWC = {rwc:.3f} -- ", end=" ")


        # Valenzuela metric
        G_ven, core_nodes = add_ideology_to_graph_22(G, allsides_scores)
        opinions = opinion_model(G_ven, core_nodes=core_nodes)
        polarization_index = get_polarization_index(opinions)[0]

        ds_dict["PI"] = polarization_index
        print(f"PI = {polarization_index:.3f}")
    else:
        print("")

    experiments.append(ds_dict)

    print("\n")

Dataset (gun): 

100%|██████████| 6913/6913 [08:05<00:00, 14.25it/s]


ECS = 0.628 --  RWC = 0.420 --  PI = 0.314


Dataset (abortion): 

100%|██████████| 5088/5088 [07:11<00:00, 11.79it/s]


ECS = 0.622 --  RWC = 0.513 --  PI = 0.186


Dataset (super_bowl): 

100%|██████████| 5463/5463 [03:48<00:00, 23.90it/s]


ECS = 0.562 --  RWC = 0.273 --  PI = 0.016


Dataset (sxsw): 

100%|██████████| 2464/2464 [03:00<00:00, 13.64it/s]


ECS = 0.465 --  RWC = 0.482 --  PI = 0.002




In [6]:
df = pd.DataFrame(experiments)

In [7]:
df

,dataset,number_of_nodes,number_of_edges,number_of_communities,echo_chamber_score,community_sizes,community_ECIs,RWC,PI
0,gun,6566,14322,2,0.627835,"[3985, 2581]","[0.6251755742357289, 0.6319404680042986]",0.419960,0.314150
1,abortion,5087,10572,2,0.621917,"[3933, 1154]","[0.6020493726635332, 0.6896280351698904]",0.512643,0.185957
2,super_bowl,5460,8732,3,0.561706,"[5346, 23, 91]","[0.5622504817995841, 0.6304746701416013, 0.512...",0.272877,0.015645
3,sxsw,2436,5325,5,0.464753,"[1568, 712, 85, 54, 17]","[0.4515471280011807, 0.419006628225444, 0.8362...",0.482049,0.001514
